In [1]:
# Importing liabraries
import numpy as np
import pandas as pd
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime
from prophet import Prophet

import joblib 

# Hyperparameter tuning
import itertools
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_cross_validation_metric
from prophet.plot import plot_components_plotly

# Enable inline plotting
%matplotlib inline

In [2]:
aep=pd.read_csv("../data/AEP_hourly.csv")
df_main = aep.copy()

In [3]:
df_main.rename(columns={'Datetime': 'ds', 'AEP_MW': 'y'}, inplace=True)
df_main

,ds,y
0,2004-12-31 01:00:00,13478.0
1,2004-12-31 02:00:00,12865.0
2,2004-12-31 03:00:00,12577.0
3,2004-12-31 04:00:00,12517.0
4,2004-12-31 05:00:00,12670.0
...,...,...
121268,2018-01-01 20:00:00,21089.0
121269,2018-01-01 21:00:00,20999.0
121270,2018-01-01 22:00:00,20820.0
121271,2018-01-01 23:00:00,20415.0


In [4]:
# Convert MW to GW
df_div=df_main.copy()
df=df_main.copy()
df_div_y=df_div['y'].div(1000)
df["y"]=df_div_y
type(df['y'][2])

numpy.float64

In [11]:
# Turn hourly data to days data
df_aux=df.copy()

# Convert "ds" column to Pandas datetime object
df_aux['ds'] = pd.to_datetime(df_aux['ds'])

# Group values by day and sum them
daily_df = df_aux.groupby(pd.Grouper(key='ds', freq='D'))['y'].sum().reset_index()

# Show the result
daily_df

,ds,y
0,2004-10-01,328.544
1,2004-10-02,311.997
2,2004-10-03,293.450
3,2004-10-04,343.417
4,2004-10-05,346.553
...,...,...
5050,2018-07-30,368.834
5051,2018-07-31,364.327
5052,2018-08-01,363.628
5053,2018-08-02,376.504


In [ ]:
# Cut data
# res = df[~(df['ds'] < '2015-01-01')]

In [14]:
df=daily_df
df.describe()

,y
count,5055.000000
mean,371.844219
std,47.605309
min,14.809000
25%,337.265500
50%,366.539000
75%,403.198500
max,548.349000


In [15]:
# Divide test and train data
train = df.iloc[:len(df) - 365]
test = df.iloc[len(df) - 365:]

In [16]:
m = Prophet()
m.fit(train)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
c:\Users\alvar\anaconda3\envs\asiot\lib\site-packages\prophet\forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


In [19]:
param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01],
    'seasonality_prior_scale': [0.01, 0.1],
    'seasonality_mode': ['additive', 'multiplicative']
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []  # Store the RMSEs for each params here

# Use cross validation to evaluate all parameters
for params in all_params:
    m = Prophet(**params).fit(df)  # Fit model with given params
    df_cv = cross_validation(m, initial='366 days', period='30 days', horizon = '30 days', parallel="processes")
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
c:\Users\alvar\anaconda3\envs\asiot\lib\site-packages\prophet\forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
INFO:prophet:Making 156 forecasts with cutoffs between 2005-10-10 00:00:00 and 2018-07-04 00:00:00
INFO:prophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x00000229B5FA2EE0>
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
c:\Users\alvar\anaconda3\envs\asiot\lib\site-packages\prophet\forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
INFO:prophet:Making 156 forecasts with cutoffs between 2005-10-10 00:0

   changepoint_prior_scale  seasonality_prior_scale seasonality_mode  \
0                    0.001                     0.01         additive   
1                    0.001                     0.01   multiplicative   
2                    0.001                     0.10         additive   
3                    0.001                     0.10   multiplicative   
4                    0.010                     0.01         additive   
5                    0.010                     0.01   multiplicative   
6                    0.010                     0.10         additive   
7                    0.010                     0.10   multiplicative   

        rmse  
0  32.905544  
1  32.962209  
2  33.001600  
3  33.097108  
4  32.395778  
5  32.375528  
6  32.483651  
7  32.490547  


In [21]:
# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
tuning_results

,changepoint_prior_scale,seasonality_prior_scale,seasonality_mode,rmse
0,0.001,0.01,additive,32.905544
1,0.001,0.01,multiplicative,32.962209
2,0.001,0.10,additive,33.001600
3,0.001,0.10,multiplicative,33.097108
4,0.010,0.01,additive,32.395778
5,0.010,0.01,multiplicative,32.375528
6,0.010,0.10,additive,32.483651
7,0.010,0.10,multiplicative,32.490547


In [20]:
# Find the best parameters
best_params = all_params[np.argmin(rmses)]
print(best_params)

{'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 0.01, 'seasonality_mode': 'multiplicative'}
